# Рекомендация тарифов

#### Описание работы :

Оператор мобильной связи выяснил: многие клиенты пользуются архивными тарифами. На основании данных о поведении клиентов, уже перешедших на новые тарифы, необходимо построить систему, способную проанализировать поведение клиентов и предложить пользователям новый тариф: «Смарт» или «Ультра».

#### Описание данных :

* `сalls` — количество звонков
* `minutes` — суммарная длительность звонков в минутах,
* `messages` — количество sms-сообщений,
* `mb_used` — израсходованный интернет-трафик в Мб,
* `is_ultra` — каким тарифом пользовался в течение месяца («Ультра» — 1, «Смарт» — 0).

#### План работы над проектом :

   1. [Обзор и загрузка данных](#Step_1)
   2. [Разделение данных на выборки](#Step_2)  
   3. [Исследование модели](#Step_3)  
   4. [Проверка модели на тестовой выборке](#Step_4) 
   5. [Проверка модели на адекватность](#Step_5) 
   6. [Общий вывод](#Step_6) 

## 1. Обзор и загрузка данных <a id="Step_1"></a>

Загрузим необходимые для работы библиотеки.

In [1]:
import pandas as pd
import math

# для разделения данных на выборки
from sklearn.model_selection import train_test_split

# для использования алгоритма классификации "решающее дерево"
from sklearn.tree import DecisionTreeClassifier

# для определения количества правильных ответов
from sklearn.metrics import accuracy_score

# для использования алгоритма классификации "случайный лес"
from sklearn.ensemble import RandomForestClassifier

# для использования алгоритма логистической регрессии
from sklearn.linear_model import LogisticRegression

# для создания простейшей модели
from sklearn.dummy import DummyClassifier

Откроем файл с данными и изучим общую информацию.

In [2]:
df = pd.read_csv('/datasets/users_behavior.csv', sep=',')

In [3]:
display(df)
df.info()
df.describe()

,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0
...,...,...,...,...,...
3209,122.0,910.98,20.0,35124.90,1
3210,25.0,190.36,0.0,3275.61,0
3211,97.0,634.44,70.0,13974.06,0
3212,64.0,462.32,90.0,31239.78,0


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
calls       3214 non-null float64
minutes     3214 non-null float64
messages    3214 non-null float64
mb_used     3214 non-null float64
is_ultra    3214 non-null int64
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


,calls,minutes,messages,mb_used,is_ultra
count,3214.000000,3214.000000,3214.000000,3214.000000,3214.000000
mean,63.038892,438.208787,38.281269,17207.673836,0.306472
std,33.236368,234.569872,36.148326,7570.968246,0.461100
min,0.000000,0.000000,0.000000,0.000000,0.000000
25%,40.000000,274.575000,9.000000,12491.902500,0.000000
50%,62.000000,430.600000,30.000000,16943.235000,0.000000
75%,82.000000,571.927500,57.000000,21424.700000,1.000000
max,244.000000,1632.060000,224.000000,49745.730000,1.000000


Как и сказано в задании, предобработка данных не понадобится. Описание самих столбцов приведено выше. Каких-либо аномалий не видно.

Тем не менее, проведём небольшую предобработку данных. Нули после точки в столбцах `calls` и `messages` нам совершенно ни к чему, поэтому приведём данные в них к целочисленному типу.

In [4]:
df[['calls', 'messages']] = df[['calls', 'messages']].astype('int')

Также, округлим значения в столбцах `minutes` и `mb_used` в большую сторону, согласно тому, как оператор взимает плату за использования траффика.

In [5]:
df['minutes'] = df['minutes'].apply(math.ceil)
df['mb_used'] = df['mb_used'].apply(math.ceil)

## 2. Разделение данных на выборки <a id="Step_2"></a>

Разделим исходные данные на обучающую, валидационную и тестовую выборки в пропорции 60% / 20% / 20% соответственно.

Сначала разделим данные в пропорции 60 / 40, сформировав тем самым обучающий набор данных.

In [6]:
df_train, df_other = train_test_split(df, test_size=0.40, random_state=12345)

Теперь, оставшиеся 40% данных, снова разделим пополам, получив тем самым валидационную и тестовую выборки в пропорции 20% каждая от исходных данных.

In [7]:
df_valid, df_test = train_test_split(df_other, test_size=0.50, random_state=12345)

Посмотрим размер получившихся выборок. 

In [8]:
print('Обучающая выборка:', df_train.shape)
print('Валидационная выборка:', df_valid.shape)
print('Тестовая выборка:', df_test.shape)

Обучающая выборка: (1928, 5)
Валидационная выборка: (643, 5)
Тестовая выборка: (643, 5)


Разделение проведено корректно.

## 3. Исследование модели <a id="Step_3"></a>

Перед тем, как приступить непосредственно к исследованию моделей, создадим переменные для хранения признаков и целевого признака каждой из выборок. Целевым признаком в нашему случае будем `is_ultra`.

Обучающая выборка. 

In [9]:
features_train = df_train.drop(['is_ultra'], axis=1)
target_train = df_train['is_ultra']

Валидационная выборка. 

In [10]:
features_valid = df_valid.drop(['is_ultra'], axis=1)
target_valid = df_valid['is_ultra']

Тестовая выборка. 

In [11]:
features_test = df_test.drop(['is_ultra'], axis=1)
target_test = df_test['is_ultra']

Переменные созданы, перейдём к исследованию моделей.

*3.1. Дерево решений.*

Итак. В нашем случае поставлена задача бинарной классификации, относящаяся к классу "обучение с учителем". Для решения данной задачи воспользуемся структурой данных, именуемой деревом решений `DecisionTreeClassifier`.

Посмотрим как будет меняться количество правильных ответов при изменении гиперпараметра, определяющего размер решающего дерева, `max_depth`.

In [12]:
for depth in range(1,16): 
    
    model = DecisionTreeClassifier(random_state=12345, max_depth= depth)
    model.fit(features_train,target_train)
    predictions_valid = model.predict(features_valid)
    print("max_depth =", depth, ": ", end='')
    print(round(accuracy_score(target_valid, predictions_valid),4))

max_depth = 1 : 0.7543
max_depth = 2 : 0.7823
max_depth = 3 : 0.7854
max_depth = 4 : 0.7792
max_depth = 5 : 0.7776
max_depth = 6 : 0.7854
max_depth = 7 : 0.7823
max_depth = 8 : 0.7776
max_depth = 9 : 0.7776
max_depth = 10 : 0.7714
max_depth = 11 : 0.7574
max_depth = 12 : 0.7636
max_depth = 13 : 0.7512
max_depth = 14 : 0.7481
max_depth = 15 : 0.7434


Я намеренно не вывожу только лучший результат, потому что хочу посмотреть на зависимость как таковую.

В итоге, при `max_depth` = 3 количество правильных ответов наиболее высокое: `accuracy` = 0.7854. Да, аналогичный результат мы получаем и на `max_depth` = 6, но совершенно точно нет смысла усложнять в ситуации, где это не принесёт никакого эффекта. Далее видно, что происходит переобучение модели, количество правильных ответов снижается, и следовательно остальные варианты нам не подходят.

*3.2. Случайный лес.*

Одна модель у нас есть. Попробуем воспользоватся алгоритмом классификации "случайный лес" (`RandomForestClassifier`) для построении модели и оценим результат.

Здесь изменяемым гиперпараметром будет количество деревьев `n_estimators`.

In [13]:
for est in range(1, 16):
    model = RandomForestClassifier(random_state=12345, n_estimators= est) 
    model.fit(features_train, target_train) 
    predictions_valid = model.predict(features_valid)
    print("n_estimators =", est, ": ", end='')
    print(round(accuracy_score(target_valid, predictions_valid),4))

n_estimators = 1 : 0.6936
n_estimators = 2 : 0.7589
n_estimators = 3 : 0.7294
n_estimators = 4 : 0.7496
n_estimators = 5 : 0.7403
n_estimators = 6 : 0.7683
n_estimators = 7 : 0.7667
n_estimators = 8 : 0.7745
n_estimators = 9 : 0.7698
n_estimators = 10 : 0.776
n_estimators = 11 : 0.7823
n_estimators = 12 : 0.7792
n_estimators = 13 : 0.7745
n_estimators = 14 : 0.7854
n_estimators = 15 : 0.7823


Также как и в предыдущем случае мы рассмотрели 15 вариантов. Лучшим оказался разультат для `n_estimators` = 14, где `accuracy` = 0.7854. На нём и остановимся. Отметим, что данный результат полностью совпадает с предыдущим алгоритмом, при том, что дерево решений требует явно меньшее количество ресурсов.

*3.3. Логистическая регрессия.*

На этот раз используем алгоритм логистической регрессии (`LogisticRegression`). 

Здесь будем рассматривать изменения для нескольких гиперпараметров: 
 - `penalty` - вид регуляризации; 
 - `C` - параметр регуляризации;
 - `tol` - указывает алгоритму оптимизации, когда следует остановиться.

Начнём с `penalty` = `l1`. 
Относительно гиперпараметра `C` поступим следующим образом. Из документации следует, что чем меньше этот гиперпараметр, тем сильнее регуляризация. Следовательно, посмотрим на изменение количества правильных ответов для данного вида регуляризация, перебрав значения параметра регуляризации `C` от 0.1 до 1.

Для того, чтобы не получать сообщение от системы о возможных проблемах `FutureWarning`, гиперпараметру `solver`, указывающему алгоритм, который используется для решения проблемы оптимизации, присвоим значение `liblinear`.

Сначала посмотрим как ведёт себя `accuracy` при установленном `penalty` и изменяющимся `C`.

In [14]:
list_c = [i / 10 for i in range(1,11,1)]

for c in list_c:
    model = LogisticRegression(random_state=12345, C = c, solver = 'liblinear', penalty = 'l1')
    model.fit(features_train, target_train) 
    predictions_valid = model.predict(features_valid)
    print("C =", c, ": ", end='')
    print(round(accuracy_score(target_valid, predictions_valid),4))

C = 0.1 : 0.7574
C = 0.2 : 0.7558
C = 0.3 : 0.7574
C = 0.4 : 0.7574
C = 0.5 : 0.7574
C = 0.6 : 0.7558
C = 0.7 : 0.7558
C = 0.8 : 0.7558
C = 0.9 : 0.7558
C = 1.0 : 0.7558


Получили максимальное количество правильных ответов при `C` = 0.1, 0.4, 0.5 . Возьмём 0.5 .

Теперь посмотрим как поведёт себя модель при изменении гиперпараметра `tol`. 
Зададим диапазон изменения от 0.00002 до 0.0004, то есть в меньшую и большую сторону от значения установленного по умолчанию (0.0001).

In [15]:
list_tol = [i / 50000 for i in range(1,21,1)]

for tol in list_tol:
    model = LogisticRegression(random_state=12345
                               , C = 0.5
                               , solver = 'liblinear'
                               , tol = tol
                               , penalty = 'l1')
    model.fit(features_train, target_train) 
    predictions_valid = model.predict(features_valid)
    print("tol =", tol, ": ", end='')
    print(round(accuracy_score(target_valid, predictions_valid),4))

tol = 2e-05 : 0.7574
tol = 4e-05 : 0.7574
tol = 6e-05 : 0.7574
tol = 8e-05 : 0.7574
tol = 0.0001 : 0.7574
tol = 0.00012 : 0.7574
tol = 0.00014 : 0.7574
tol = 0.00016 : 0.7574
tol = 0.00018 : 0.7543
tol = 0.0002 : 0.7543
tol = 0.00022 : 0.7543
tol = 0.00024 : 0.7543
tol = 0.00026 : 0.7543
tol = 0.00028 : 0.7543
tol = 0.0003 : 0.7543
tol = 0.00032 : 0.7543
tol = 0.00034 : 0.7543
tol = 0.00036 : 0.7543
tol = 0.00038 : 0.7527
tol = 0.0004 : 0.7527


Из полученных результатов хорошо видно, что после того, как значение остановки алгоритма превышает 0.00016 происходит уменьшение количества правильных ответов. 

Таким образом, для гиперпарамера `penalty` = `l1` лучший результат, равный 0.7574, можно получить при `C` = 0.5 и `tol` = 0.0016.

Создадим соответсвующую сводную таблицу для простоты сравнения результатов изменения гиперпараметров для данного алгоритма.

In [16]:
df_gip_LR = pd.DataFrame({'penalty': 'l1'
                          , 'C': 0.5
                          ,'tol':0.0016
                          , 'accuracy': 0.7574}, index=[0])

In [17]:
df_gip_LR

,penalty,C,tol,accuracy
0,l1,0.5,0.0016,0.7574


Чтобы в следуюший раз нам не разбивать выбор гиперпараметров `tol` и `C` при заданном `penalty` на несколько операций, напишем функцию, которая рассчитает нам лучшую комбинацию.

Для проверки корректности работы функции, используяем данные рассчитанного выше варианта.

In [18]:
def best_result_LG (penalty, solver):

    list_c = [i / 10 for i in range(1,11,1)]
    list_tol = [i / 50000 for i in range(1,21,1)]

    list_res = []
    list_c_res = []
    list_tol_res = []

    for c in list_c:
        for tol in list_tol:
            model = LogisticRegression(random_state=12345
                                       , penalty = penalty
                                       , C = c
                                       , tol = tol
                                       , solver = solver)
                                       
            model.fit(features_train, target_train) 
            predictions_valid = model.predict(features_valid)
            result = round(accuracy_score(target_valid, predictions_valid),4)
        
            list_res.append(result)
            list_c_res.append(c)
            list_tol_res.append(tol)
        

    df_gip = pd.DataFrame({'penalty': penalty
                           , 'C': list_c_res
                           ,'tol':list_tol_res
                           ,'accuracy': list_res})

    return df_gip.query('accuracy == accuracy.max()')

In [19]:
best_result_LG('l1', 'liblinear')

,penalty,C,tol,accuracy
1,l1,0.1,0.00004,0.7574
2,l1,0.1,0.00006,0.7574
3,l1,0.1,0.00008,0.7574
4,l1,0.1,0.00010,0.7574
5,l1,0.1,0.00012,0.7574
6,l1,0.1,0.00014,0.7574
7,l1,0.1,0.00016,0.7574
20,l1,0.2,0.00002,0.7574
40,l1,0.3,0.00002,0.7574
41,l1,0.3,0.00004,0.7574


Как видим, возможных комбинаций, при которых будет получен наилучший результат, достаточно много. Есть и наш результат, полученный выше с помощью нескольких операций изменения гиперпараметров.

Изменим функцию так, чтобы она выводила только один лучший результат.

In [20]:
def best_result_LG (penalty, solver):

    list_c = [i / 10 for i in range(1,11,1)]
    list_tol = [i / 50000 for i in range(1,21,1)]

    list_res = []
    list_c_res = []
    list_tol_res = []

    for c in list_c:
        for tol in list_tol:
            model = LogisticRegression(random_state=12345
                                       , penalty = penalty
                                       , C = c
                                       , tol = tol
                                       , solver = solver)
                                       
            model.fit(features_train, target_train) 
            predictions_valid = model.predict(features_valid)
            result = round(accuracy_score(target_valid, predictions_valid),4)
        
            list_res.append(result)
            list_c_res.append(c)
            list_tol_res.append(tol)
        

    df_gip = pd.DataFrame({'penalty': penalty
                           , 'C': list_c_res
                           ,'tol':list_tol_res
                           ,'accuracy': list_res})

    return df_gip.query('accuracy == accuracy.max()').head(1)

In [21]:
best_result_LG('l1', 'liblinear')

,penalty,C,tol,accuracy
1,l1,0.1,0.00004,0.7574


Отлично, теперь посмотрим, как будет меняется количество правильных ответов, в зависимости от разных вариантов гиперпараметра `penalty`.

Возьмем `penalty` = `l2`

In [22]:
df_gip_LR = pd.concat([df_gip_LR, best_result_LG('l2', 'liblinear')])

In [23]:
df_gip_LR

,penalty,C,tol,accuracy
0,l1,0.5,0.00160,0.7574
143,l2,0.8,0.00008,0.7589


`Accuracy` незначительно, но подрос.

Таким образом, исходя из полученных результатов, для логистической регресии примем следующие гиперпараметры: 
  - `penalty` = `l2`
  - `C` = 0.8
  - `tol` = 0.00008

## 4. Проверка модели на тестовой выборке <a id="Step_4"></a>

После того, как мы определились с гиперпараметрами наших алгоритмов, оценим количество правильных ответов для каждой из моделей на тестовой выборке.

*4.1. Дерево решений.*

In [24]:
model = DecisionTreeClassifier(random_state=12345, max_depth = 3)
model.fit(features_train,target_train)
predictions_test = model.predict(features_test)
result = round((accuracy_score(target_test, predictions_test)),5)
print('Accuracy =', result)

Accuracy = 0.77916


*4.2. Случайный лес.*

In [25]:
model = RandomForestClassifier(random_state=12345, n_estimators= 14) 
model.fit(features_train, target_train) 
predictions_test = model.predict(features_test)
result = round(accuracy_score(target_test, predictions_test),5)
print('Accuracy =', result)

Accuracy = 0.79782


*4.3. Логистическая регрессия.*

In [26]:
model = LogisticRegression(random_state=12345
                           , penalty = 'l2'
                           , C = 0.8
                           , tol = 0.00008
                           , solver = 'liblinear')
model.fit(features_train, target_train) 
predictions_test = model.predict(features_test)
result = round(accuracy_score(target_test, predictions_test),5)
print('Accuracy =', result)

Accuracy = 0.74028


Итак. Больше всего правильных предсказаний (`accuracy` = 0.79782) нам даёт модель, использующая алгоритм "случайный лес". Её и примем за основную.

## 5. Проверка модели на адекватность <a id="Step_5"></a>

Чтобы проверить полученную модель на адекватность сравним её по параметру `accuracy` с простейшей (dummy) моделью, которая всегда предсказывает наиболее часто встречающийся класс.

In [27]:
dummy_clf = DummyClassifier(strategy = "most_frequent", random_state=0)

dummy_clf.fit(features_train, target_train)

result = round((dummy_clf.score(features_test, target_test)), 5)

print('Accuracy =', result)

Accuracy = 0.68429


Результат существенно ниже, чем у выбранной нами модели. Следовательно, можно сделать вывод, что обученная ранее модель вполне адекватна.

## 6.  Общий вывод <a id="Step_6"></a>

В ходе исследования нам удалось получить следующие результаты для каждого из трёх алгоритмов классификации: 

- дерево решений `DecisionTreeClassifier`, `Accuracy` = 0.77916 
- случайный лес `RandomForestClassifier`, `Accuracy` = 0.79782  
- логистическая регрессия `LogisticRegression`, `Accuracy` = 0.74028  


На основании приведённых выше результатов можно сделать вывод о том, что модель, построенная на алгоритме "случайный лес" `RandomForestClassifier` с гиперпараметрами `penalty` = l2, `C` = 0.8, `tol` = 0.00008, даёт наибольшее количество правильных ответов, где `Accuracy` = 0.79782, и как следствие именно её стоит рекомендовать оператору мобильной связи для решения задачи классификации - выбора подходящего тарифа.  